In [1]:
import pandas as pd
import numpy as np
import altair as alt
import json
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
# with open('/Users/arnav/Downloads/answers.json') as f:
#     data = json.load(f)
#     # data = data[6:]
#     data = data[11:]

In [3]:
with open('/Users/arnav/Downloads/answers2.json') as f:
    data = json.load(f)

In [4]:
data[0].keys()

dict_keys(['_id', 'parsed_params', 'survey', 'order', 'study_end_timestamp', 'study_start_timestamp', '__v'])

In [6]:
# item_df['question_image'].nunique()

## Preprocessing Data

In [7]:
tests = ['wainer', 'brbf', 'ggr-mc', 'vlat', 'calvi', 'practice']

item_df = []
for test in tests:
    idf = pd.read_csv(f'https://data-visualization-benchmark.s3.us-west-2.amazonaws.com/{test}/questions.csv')
    idf['test_type'] = test
    item_df.append(idf)

item_df = pd.concat(item_df)
item_df['question_image'] = item_df['question'] + " + " + item_df['image_file']
item_df = item_df.drop(columns=['question', 'image_file'])

In [8]:
temp_practice_image_map = {
    "The grid shows the location of several buildings. Which building is located at B3?": "practice0.png",
    "Aubery made a tower with 5 layers of cubes, as shown. She then removed the top 2 layers of the tower. How many cubes did she remove?": "practice1.png",
    "What number is represented by point X on the number line?": "practice2.png",
    "The first six number of the patterns are shown. The pattern continues by adding the same number each time. What is the next number in the pattern?": "practice3.png",
    "Determine whether the statement is True or False.": "practice4.png",
}

In [9]:
def generate_feedback_data(data):
    trials_df = []
    
    for row in data:
        try:
            if "survey" in list(row.keys()):
                participant_trial = row['survey']
                participant_trial['participant_id'] = row['parsed_params']['PROLIFIC_ID']
                participant_trial['courses_taken_count'] = len(participant_trial['classesTaken'].keys())
                trials_df.append(participant_trial)
        except:
            pass
    
    trials_df = pd.DataFrame(trials_df)

    return trials_df

In [10]:
def generate_trial_data(data):
    trials_df = []
    
    for row in data:
        try:
            participant_trial = list(row['order']['order'].values())[0]
            # print(participant_trial.keys(), participant_trial['task_id'])
            participant_trial['study_timestamp'] = row['order']['timestamp']
            participant_trial['participant_id'] = row['order']['params']['PROLIFIC_ID']
            participant_trial['PROLIFIC_ID'] = row['order']['params']['PROLIFIC_ID']
            participant_trial['SESSION_ID'] = row['order']['params']['SESSION_ID']
            participant_trial['STUDY_ID'] = row['order']['params']['STUDY_ID']
            participant_trial['time_spent'] = round(participant_trial['time_elapsed'] / 1000, 3)
            participant_trial['history_length'] = len(participant_trial['optionHistory'])
            if 'practice' in participant_trial['order_id']:
                participant_trial['image_file'] = temp_practice_image_map[participant_trial['question']]
                
            if participant_trial['participant_id'] in ["67490943c58dd32e763b6c75", "675b9a072275e60fc54518e6"]:
                continue
            trials_df.append(participant_trial)
            
        except:
            try:
                if row['order']['params']['SESSION_ID'] == "6758b92dec146e0f7ef14085":
                    print("aaa")
            except:
                pass
            pass
    
    trials_df = pd.DataFrame(trials_df)
    trials_df = trials_df[[
        'order_id',
        'study_timestamp',
        'partition_group', 
        'question', 
        'image_file', 
        'participant_id', 
        'time_spent', 
        'answer', 
        'history_length', 
        'optionHistory',
        'seed',
        'participant_cond_id'
    ]]
    # trials_df = trials_df.dropna()
    trials_df = trials_df[trials_df['participant_id'] != '12'] # omitting ID for testing
    trials_df['question_image'] = trials_df['question'] + " + " + trials_df['image_file']
    df = pd.merge(item_df, trials_df, on='question_image')
    df['is_correct'] = df.apply(lambda r : int(r['correct_answer'] == r['answer']), axis=1)
    return df, trials_df


with open('/Users/arnav/Downloads/answers2.json') as f:
    data = json.load(f)
    print("Length of database: ", len(data))
    
df, trials_df = generate_trial_data(data)
df['order_id_int'] = df['order_id'].apply(lambda o : o.replace('order', '').replace('practice', '')).astype(int)

Length of database:  28054


In [11]:
# df['participant_cond_id'].unique()

In [12]:
# p_df = df[['participant_id']].value_counts()

# non_practice_df = df[df['test_type'] != 'practice'].copy()
# non_practice_df['order_id_int'] = non_practice_df['order_id'].apply(lambda o : o.replace('order', '')).astype(int)

# non_practice_df = non_practice_df.drop_duplicates(subset=['question_image', 'participant_id', 'order_id'])
# non_practice_df = non_practice_df.dropna(subset=['seed', 'participant_cond_id', 'partition_group'])

# participant_response_counts = non_practice_df[['participant_id']].value_counts().reset_index()
# valid_participants = participant_response_counts[participant_response_counts['count'] == 46]['participant_id']

# complete_df = non_practice_df[non_practice_df['participant_id'].isin(valid_participants)]

In [13]:
# seed_participant_count = complete_df[['participant_cond_id']].value_counts().reset_index()
# seed_participant_count['participant_cond_id'] = seed_participant_count['participant_cond_id'].astype(int) 

# print(seed_participant_count['participant_cond_id'].max())
# all_missing_sets = set(range(174)) - set(seed_participant_count['participant_cond_id'].to_list())
# all_missing_sets

In [16]:
# for missing in all_missing_sets:
#     pdf = df[(df['participant_cond_id'] == str(missing))]
#     missings_qs = set(range(1, 47)) - set(pdf['order_id_int'])
#     if len(missings_qs) != 46:
#         print(missing, missings_qs, pdf['participant_id'].unique())
    # else:
    #     print(missing)

# pdf = non_practice_df[non_practice_df['participant_id'] == '66cbc2627559d53c0e3e4ac4']
# all_missing_sets = 

# print(all_missing_sets)
# pdf[pdf['order_id_int'] == 22][['time_spent', 'optionHistory', 'partition_group', 'history_length', 'test_type', 'participant_id']]

In [17]:
# pdf[pdf['order_id_int'] == 22][['time_spent', 'optionHistory', 'history_length', 'test_type', 'participant_id']]

In [2757]:
# df[df['participant_id'] == '67941592810ec75beebe7fa4']['order_id'].value_counts()

In [18]:
def generate_complete_trial_data(data):
    trials_df = []
    
    for row in data:
        try:
            if row['parsed_params']['PROLIFIC_ID'] == "67941592810ec75beebe7fa4":
                continue
            #     print('aa')
            participant_id = row['parsed_params']['PROLIFIC_ID']
            
            
            for key in row['order'].keys():
                item = row['order'][key]
                item['order_id'] = key
                item['participant_id'] = participant_id
                item['image_file'] = item['image_link'].split("/")[-1]
                item['time_spent'] = round(item['time_elapsed'] / 1000, 3)
                item['history_length'] = len(item['optionHistory'])
                
                trials_df.append(item)
        except:
            # if 'parsed_params' not in row.keys():
            #     print(row['order'].keys())
            #     print(row['params']
            #     print(row.keys(), row['study_start_timestamp'])
            
            pass

    trials_df = pd.DataFrame(trials_df)
    trials_df = trials_df[[
        'order_id',
        'question', 
        'partition_group', 
        'image_file',
        'participant_id', 
        'time_spent', 
        'answer', 
        'history_length', 
        'optionHistory',
        'participant_cond_id'
    ]]
    trials_df = trials_df[trials_df['participant_id'] != '12'] # omitting ID for testing
    trials_df['question_image'] = trials_df['question'] + " + " + trials_df['image_file']
    df = pd.merge(item_df, trials_df, on='question_image')
    df['is_correct'] = df.apply(lambda r : int(r['correct_answer'] == r['answer']), axis=1)
    return df

complete_data = generate_complete_trial_data(data)
complete_data['order_id_int'] = complete_data['order_id'].apply(lambda o : o.replace('order', '').replace('practice', '')).astype(int)

In [30]:
combined_df = pd.concat([complete_data, df[df['study_timestamp'] > 1737449330]])

practice = combined_df['participant_id'].nunique()
non_practice = combined_df[combined_df['test_type'] != 'practice']['participant_id'].nunique()

print("Failed practice: ", practice - non_practice)
print("Failed practice: ", practice - (practice - non_practice))

Failed practice:  37
Failed practice:  466


In [42]:
# item_df[['test_type', 'question']].value_counts()
item_df['question'] = item_df['question_image'].apply(lambda i : i.split(" + ")[0])

In [45]:
item_df[item_df['test_type'] == 'brbf']['question'].nunique()

44

In [2759]:
# {35, 39} ['67941df3f640625fcdf46333']
# {45} ['67941e2bad82afc6bc2b0de9']
# {43, 44, 45, 46} ['67941593f8c4fac723882876']
# {30} ['67941592810ec75beebe7fa4']
# len(complete_data)
# complete_data[['participant_id']].value_counts()
# set(range(50)) - set(complete_data[['participant_cond_id']].value_counts().reset_index()['partition_group'])
# ids = complete_data[['participant_cond_id']].value_counts().reset_index()
# ids['participant_cond_id'] = ids['participant_cond_id'].astype(int)
# print(ids['participant_cond_id'].max())
# set(range(117)) - set(ids['participant_cond_id'])

In [2760]:
# complete_data['participant_cond_id'].unique()

In [2777]:
combined_df = pd.concat([complete_data, df[df['study_timestamp'] > 1737449330]])
completed_df = combined_df[combined_df['test_type'] != 'practice'].drop_duplicates(subset=['participant_id', 'order_id'])
completed_df['ptrial_id'] = completed_df['participant_id'] + "--" + completed_df['participant_cond_id']
completed_df = completed_df.dropna(subset=['participant_cond_id'])
completed_df['participant_cond_id'] = completed_df['participant_cond_id'].astype(int)

trial_count = completed_df[['ptrial_id']].value_counts().reset_index()
trial_count = trial_count[trial_count['count'] < 23]['ptrial_id']
completed_df = completed_df[~completed_df['ptrial_id'].isin(trial_count)]

item_counts = completed_df[['question_image']].value_counts().reset_index()['count']
participant_counts = completed_df[['participant_id']].value_counts().reset_index()['count']
participant_cond_counts = completed_df['participant_cond_id'].unique()

# below 80
item_ids_df = completed_df[['question_image']].value_counts().reset_index()
item_under_80 = item_ids_df[item_ids_df['count'] < 80]
item_80_over = item_ids_df[item_ids_df['count'] >= 80]

print(f'participant count {completed_df["participant_id"].nunique()}')
print(f"max participant item count: {participant_counts.max()}, min participant item count: {participant_counts.min()}")
print(f"max items collected count: {item_counts.max()}, min items collected count: {item_counts.min()}")
print(f"max condition idx: {participant_cond_counts.max()}, min condition idx: {participant_cond_counts.min()}")
print(f"items with less that 80 counts: {len(item_under_80)}, and 80 or more count: {len(item_80_over)}")

# completed_df.to_csv("participant_responses.csv")


# trial_count[trial_count['count'] > 23]
# completed_df #[['participant_cond_id', 'participant_id']].value_counts().reset_index()
# completed_df[['participant_id', 'participant_cond_id',]].value_counts().reset_index().sort_values(by='count', ascending=False)
# trial_count
# print(item_counts_counts.sort_values())
# item_counts #.sort_values().value_counts()


participant count 426
max participant item count: 46, min participant item count: 27
max items collected count: 92, min items collected count: 80
max condition idx: 397, min condition idx: 0
items with less that 80 counts: 0, and 80 or more count: 230


In [ ]:
# 672f17fa54603fd2c8d49cb7

In [2747]:
# completed_df[['participant_id']].drop_duplicates().to_csv("./pids.csv", index=False)
# !open .


In [2776]:
msising_idss = df[df['participant_id'] == '6782604111204dacf170e329']

print(len(msising_idss[msising_idss['test_type'] != 'practice']))
msising_idss[msising_idss['test_type'] != 'practice']

0


,Unnamed: 0.1,Unnamed: 0,mc1,mc2,mc3,mc4,correct_answer,chart_type,task_category,image_link,...,image_file,participant_id,time_spent,answer,history_length,optionHistory,seed,participant_cond_id,is_correct,order_id_int


In [2711]:
# df[['participant_cond_id']].value_counts().reset_index()['count'].unique()
# completed_df[['participant_cond_id']].value_counts().reset_index()
# item_counts.sort_values()
# completed_df.sort_values(by='timestamp')

In [2712]:
values = [t+75 for t in [
    0, 7, 15, 23, 28, 31, 32, 33, 34, 37, 39, 43, 46, 58, 71, 83, 85, 87, 92, 98,
  100, 101, 102, 103, 104, 108, 112, 114, 118, 120, 121, 122, 128, 130, 131,
  132, 136, 141, 144, 148, 152, 159, 165, 166, 168, 169, 171, 174, 177, 181,
  182, 184, 185, 187, 189, 194, 199, 206, 207, 208, 214, 219, 225, 226, 227,
  230,]
]

In [2713]:
values2 = [
    0, 15, 32, 33, 39, 83, 85, 100, 102, 103, 112, 114, 120, 122, 128, 130, 132,
  136, 168, 171, 177, 182, 184, 185, 189, 206, 207, 208, 214, 219, 225, 226,
  227, 230, 233, 236, 237, 238, 241, 242, 246, 248, 249, 250, 252, 253, 256,
  257, 260, 261, 263, 264, 266, 267, 268, 270, 271, 274, 277, 279, 280, 282,
  288, 289, 292, 296, 300, 302, 305, 306, 307, 308, 309, 311, 312, 313, 314,
  315, 317, 320, 321, 322, 323, 324
]

In [2714]:
missing_cond_ids = list(set(range(400)) - set(completed_df['participant_cond_id']))
missing_cond_ids = [id-75 for id in missing_cond_ids]
missing_cond_ids.sort()
# print(len(missing_cond_ids))
# missing_cond_ids

# participant count 346
# max participant item count: 46, min participant item count: 27
# max items collected count: 78, min items collected count: 59
# max condition idx: 394, min condition idx: 0

# missing_cond_ids
# len(set(values2) - set(missing_cond_ids))

In [2715]:
missing_cond_ids

[264, 312, 313, 314, 315, 320, 323, 324]

In [2717]:
missing_cond_ids

[264, 312, 313, 314, 315, 320, 323, 324]

In [2718]:
# print(completed_df)
# df[~df['participant_id'].isin(completed_df['participant_id'])]

In [2719]:
# completed_df[['participant_id']].value_counts().reset_index()

In [2720]:
non_practice = df[df['test_type'] != 'practice']
count_df = df[df['test_type'] != 'practice'][['participant_id']].value_counts().reset_index()
# count_df[count_df['count'] != 46]

In [2721]:
# non_practice[non_practice['participant_id'] == "6732eac7f92b3be7edca9b89"].drop_duplicates(subset=['participant_id', 'order_id'])

In [2722]:
# df[df['participant_id'] == '677e75b259b7f06d0c32d86e'].drop_duplicates(subset=['order_id'])[['question_image', 'answer']].value_counts().reset_index()

In [2723]:
exp_df = pd.read_csv("/Users/arnav/Desktop/vt-fusion/stimuli/experiments.csv")
exp_df['question_image'] = exp_df['question'] + " + " + exp_df['image_file']

In [2725]:
# 327-75

In [2726]:
# exp_df[
#     (exp_df['participant_cond_id'].isin([327, 339, 387, 388, 389, 390, 395, 398, 399])) &
#     (
#         (exp_df['question_image'] == 'True or False: Monthly sales for store B is increasing from Jan to Dec. + T52.png') |
#         (exp_df['question_image'] == 'Which group of 2 planets (either green, red, or orange) has the biggest difference in life expectancy? + bg-table-6.png')
#     )
# ]

In [1574]:
trial_count = completed_df[['participant_cond_id', 'participant_id']].value_counts().reset_index()
trial_count = trial_count[~trial_count['participant_id'].isin(incomplete_participants)]
trial_count['participant_cond_id'] = trial_count['participant_cond_id'].astype(int)

trial_count = trial_count[trial_count['count'] == 46]
trials_completed = set(trial_count.sort_values(by="participant_cond_id")['participant_cond_id'].unique())
all_trials = set(range(106))
all_trials - trials_completed

{75, 77, 79, 82, 90, 96, 98, 101, 103}

In [1575]:
# completed_df[['participant_cond_id']].value_counts()

In [1576]:
trial_count = completed_df[['participant_cond_id', 'participant_id']].value_counts().reset_index()
trial_count = trial_count[~trial_count['participant_id'].isin(incomplete_participants)]
trial_count['participant_cond_id'] = trial_count['participant_cond_id'].astype(int)
trial_count[trial_count['count'] != 46].sort_values(by='participant_cond_id', ascending=False)

# ['participant_id'] 

,participant_cond_id,participant_id,count
106,116,67559dc628f6cc5165666711,45
104,113,67856301b6b101f746eb0e1e,45
107,101,677160ccb75e331d9ac70394,45
108,96,67600da0497fab54effd903e,44
105,79,66a7fe579f334f72a2b3e59c,45
112,77,6758b92dec146e0f7ef14085,42
113,73,67352ce06354a65a2a47a063,42
103,73,672a61b2e2386206173cf1e0,45
119,73,64aff8a2c8ddc83286a349c3,4
110,72,676e9c991ee26c2d2c0c66e9,44


In [1358]:
# aa = completed_df[['participant_cond_id', 'participant_id']].value_counts().reset_index()
# completed_df[completed_df['participant_id'] == '676a9ff29f682327702b7b06'][['participant_cond_id', 'participant_id', 'seed', 'question_image']]

In [1359]:
# m = completed_df[completed_df['participant_id'] == '675434b1ee2fa6f30842fe1e'][['order_id']].value_counts()
# m 

In [1360]:
trial_count[trial_count['count'] != 46]

,participant_cond_id,participant_id,count
101,20,676a9ff29f682327702b7b06,45
102,20,677ae74330d6b6bc97867206,45
103,73,672a61b2e2386206173cf1e0,45
104,113,67856301b6b101f746eb0e1e,45
105,79,66a7fe579f334f72a2b3e59c,45
106,116,67559dc628f6cc5165666711,45
107,101,677160ccb75e331d9ac70394,45
108,96,67600da0497fab54effd903e,44
109,22,6780e43290a835667fac9d71,44
110,72,676e9c991ee26c2d2c0c66e9,44


In [1361]:
a = df[df['participant_id'] == '67352ce06354a65a2a47a063'][['question_image', 'test_type']].value_counts().reset_index()
a[a['test_type'] == 'practice']

,question_image,test_type,count
0,What number is represented by point X on the n...,practice,2
1,The grid shows the location of several buildin...,practice,2
2,"Aubery made a tower with 5 layers of cubes, as...",practice,2
3,Determine whether the statement is True or Fal...,practice,2
15,The first six number of the patterns are shown...,practice,1


In [1362]:
# a.groupby('test_type')['is_correct'].mean()
a

,question_image,test_type,count
0,What number is represented by point X on the n...,practice,2
1,The grid shows the location of several buildin...,practice,2
2,"Aubery made a tower with 5 layers of cubes, as...",practice,2
3,Determine whether the statement is True or Fal...,practice,2
4,"Which season has more rain, the summer or the ...",wainer,1
5,What is the average adult literacy on the 5 pl...,brbf,1
6,The global smartphone market share of Apple is...,vlat,1
7,The number of unique visitors for Amazon was m...,vlat,1
8,There is a negative linear relationship betwee...,vlat,1
9,True or False: The annual number of trees plan...,calvi,1


In [1363]:
df[df['participant_id'] == '6780e43290a835667fac9d71']['time_spent'].mean()

25.936294871794868

In [1364]:
df[df['participant_id'] == '678c09c3eefc2d7fd855b6e8'].groupby('test_type')['is_correct'].mean()

test_type
practice    0.8
Name: is_correct, dtype: float64

In [1365]:
feedback_df[feedback_df['participant_id'] == '6780e43290a835667fac9d71']['experience'].unique()

array(['very cool'], dtype=object)

In [1366]:
feedback_df[['experience']].value_counts().reset_index()

,experience,count
0,,32
1,no,3
2,N/A,2
3,That was fun! Instructions very clear for me.,1
4,The first parts (the basic charts with just ra...,1
...,...,...
62,"Nothing was unclear. On a few occasions, I tho...",1
63,One of the graphs had a question about unemplo...,1
64,Perhaps less time between the question appeari...,1
65,Some graphs were harder to read.,1


## Participant / Item Sanity Checks

In [1367]:
print(f"Total number of unique items: {df['question_image'].nunique()} / {item_df['question_image'].nunique()}" )

Total number of unique items: 235 / 235


### Questions covered by each participant

In [1368]:
# df[['question_image']].value_counts()

In [1369]:
# item_count = df[df['test_type'] != 'practice']['question_image'].value_counts().reset_index()
# item_count[item_count['count'] > 1]
len_df_all = len(df[df['test_type'] != 'practice'])
len_df_parsed = len(df[df['test_type'] != 'practice'].drop_duplicates(subset=['participant_id', 'order_id']))
print(len_df_parsed, len_df_all)

6447 6506


In [1370]:
completed_df = df[df['test_type'] != 'practice'].drop_duplicates(subset=['participant_id', 'order_id'])

participant_test_accuracy = completed_df.groupby(['participant_id', 'test_type'])['is_correct'].mean().reset_index()
participant_test_accuracy = participant_test_accuracy.pivot(index='participant_id', columns='test_type', values='is_correct').reset_index()

participant_stats = completed_df.groupby(['participant_id', 'partition_group']).agg(
    items_completed=('partition_group', 'size'),
    total_time_min=('time_spent', lambda t : np.sum(t) / 60),
    avg_time_sec=('time_spent', lambda t : np.mean(t)),
    prop_correct=('is_correct', 'mean'),
).reset_index()

pd.merge(participant_stats, participant_test_accuracy, on='participant_id').sort_values(['partition_group'])

,participant_id,partition_group,items_completed,total_time_min,avg_time_sec,prop_correct,brbf,calvi,ggr-mc,vlat,wainer
146,679315ca612887368da09ccb,0,46,27.161450,35.427978,0.630435,0.571429,0.583333,0.666667,0.636364,0.833333
72,66c0df311f98d7c9661669d9,0,46,19.042917,24.838587,0.630435,0.785714,0.500000,0.333333,0.545455,0.833333
123,677ae74330d6b6bc97867206,0,45,24.435367,32.580489,0.644444,0.785714,0.500000,1.000000,0.454545,0.800000
71,66be7f34388490c8fca8b77e,0,46,27.139300,35.399087,0.608696,0.714286,0.333333,0.000000,0.727273,1.000000
68,66abd9806349818b3dab4dc3,0,46,19.247483,25.105413,0.826087,1.000000,0.583333,0.666667,0.818182,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
81,67167054eb62d70b4149a77d,4,46,16.543467,21.578435,0.760870,0.928571,0.500000,1.000000,0.818182,0.714286
57,65fb580ab7fbb2d518ee8eaa,4,46,16.975283,22.141674,0.543478,0.785714,0.250000,0.500000,0.545455,0.571429
65,66672c0098c293e2b2718da8,4,46,17.893217,23.338978,0.760870,0.785714,0.416667,1.000000,1.000000,0.857143
30,614d673e3a5fd5cb2b5b6eb7,4,46,14.078433,18.363174,0.608696,0.500000,0.416667,0.500000,0.818182,0.857143


In [1371]:
participant_stats[['partition_group']].value_counts()

partition_group
0                  34
1                  30
3                  29
2                  27
4                  27
Name: count, dtype: int64

In [1372]:
# participant_stats[['seed', 'partition_group']].value_counts()

In [1373]:
print("Number of questions timed out: ", len(completed_df[completed_df['time_spent'] >= 60]))

Number of questions timed out:  197


In [1374]:
# missing question (should be 0)
item_df[~item_df['question_image'].isin(df['question_image'])]

,Unnamed: 0.1,Unnamed: 0,mc1,mc2,mc3,mc4,correct_answer,chart_type,task_category,image_link,...,graph_type,Unnamed: 12,misleader_type,misleading_answer,misleading_answer_2,task_category_2,task_category_3,correct_answer_2,Unnamed: 10,question_image


In [1375]:
df_count = df[df['test_type'] != 'practice']['question_image'].value_counts().reset_index()

# df_count[df_count['count'] != 6]

In [1376]:
# completed_df[
# (completed_df['participant_id'] == '67490943c58dd32e763b6c75') & 
# (completed_df['test_type'] == 'wainer')
# ]

In [1377]:
# participant_stats

### Feedback

In [1379]:
feedback_df = generate_feedback_data(data)
feedback_df = feedback_df[feedback_df['participant_id'].apply(lambda pid : pid in df['participant_id'].unique())]
feedback_df #[['participant_id', 'experience']].to_dict(orient='records') #[41] #.tolist() #.loc[26].tolist()

,education,graphMakingFrequency,experience,classesTaken,participant_id,courses_taken_count,colorBlindness
21,ba,3,no,{'Differential Equations': True},66be7f34388490c8fca8b77e,1,yes
22,ms,4,N/A,{'Data Science': True},67208d9a951b62f26d592142,1,no
23,ma,3,The instructions were clear.,"{'Probability and Statistics': True, 'Real Ana...",6732eac7f92b3be7edca9b89,7,no
24,sc,2,,"{'Algebra': True, 'Geometry': True, 'Computer ...",652ab3f37d5885fc2d4a9240,4,no
25,ms,4,NONE,{'Real Analysis': True},67490943c58dd32e763b6c75,1,no
...,...,...,...,...,...,...,...
154,ba,2,it was unclear to me at a moment until i under...,"{'Linear Algebra': True, 'Probability and Stat...",6741b2d133472ff2947350bf,6,no
155,ms,3,Easy and the instructions were cleared.,"{'Algebra': True, 'Geometry': True, 'Calculus'...",677160ccb75e331d9ac70394,11,no
156,sc,3,I'm not sure if the instructions stated how ma...,"{'Geometry': True, 'Algebra': True, 'Trigonome...",65654bb65a81489784483bf0,8,no
157,bs,3,Time should be extended,"{'Probability and Statistics': True, 'Linear A...",67856301b6b101f746eb0e1e,11,yes


In [1380]:
feedback_df[feedback_df['participant_id'] == '67365d664380cdd36e1afe61']['experience'].unique()

array(['Generally, the task was moderately challenging, especially the timed screener questions. However, I found it very engaging. The instructions were all clear and I did not experience any technical difficulties.'],
      dtype=object)

In [1381]:
feedback_df['experience'].unique()

array(['no', 'N/A', 'The  instructions were clear.', '', 'NONE',
       'Nothing was unclear. On a few occasions, I thought I had selected the right answer, but I was wrong. Would appreciate feedback (i.e. the correct answers to missed questions) at the end of the study to understand why I was wrong! I was never strong at math, but this task was not as daunting as I thought it would be. Wishing you continued success with your work!',
       'no issues', 'everything was clear and fun',
       'I think this was a good mix between hard and easy. I felt like not being from this profession that I was still able to have a good chance at getting questions right based on my past knowledge.',
       'This was a very well-done experiment and easy to follow. It could be difficult at times but overall was very original and creative in its presentation on the Google Chrome internet browser. I found the instructions to be very clear and easy to understand, and I did not have any technical difficulti

### All participant attempts

In [319]:
# df

In [535]:
# unique_pids = df.unique()
practice_stats = df[df['test_type'] == 'practice'].groupby(['participant_id',]).agg(
    practice_items_completed=('participant_id', 'size'),
    practice_total_time_min=('time_spent', lambda t : np.sum(t) / 60),
    practice_avg_time_sec=('time_spent', lambda t : np.mean(t)),
    prop_correct=('is_correct', 'mean'),
    avg_options_length=('history_length', lambda h: np.mean(h))
).reset_index()

practice_stats['completed'] = practice_stats['participant_id'].apply(
    lambda pid : pid in participant_stats['participant_id'].unique()
)

practice_stats

,participant_id,practice_items_completed,practice_total_time_min,practice_avg_time_sec,prop_correct,avg_options_length,completed
0,53976390fdf99b73530fe503,10,2.075950,12.4557,0.90,1.0,True
1,5685775f333cbd000c4e4c27,5,1.799233,21.5908,1.00,1.0,True
2,5b15307351caaf0001228d9b,1,1.007350,60.4410,0.00,0.0,False
3,5b16e1a5457ace0001b40d46,5,0.907733,10.8928,1.00,1.4,True
4,5b72d3f8afce7600019fe275,10,2.753017,16.5181,0.90,1.0,True
...,...,...,...,...,...,...,...
67,678807fe229dfba3630cecd2,20,8.177100,24.5313,0.85,0.9,True
68,6788cde09959550697be5b6b,10,4.009850,24.0591,0.90,1.0,True
69,678c76ff10d7de60d4a4fd56,5,3.058517,36.7022,1.00,1.0,True
70,678d702b1b04e7a307fdfde5,10,2.921417,17.5285,0.90,1.0,True


### Questions Completed 2 times

In [375]:
participant_stats

,participant_id,partition_group,items_completed,total_time_min,avg_time_sec,prop_correct
0,5b16e1a5457ace0001b40d46,0,46,16.088950,20.985587,0.608696
1,5dfdc45e617219a8e2e05c26,3,46,12.233183,15.956326,0.673913
2,60fd6618f9a377f54dcdf318,1,46,12.583050,16.412674,0.673913
3,6294cced417fa0a733ba14a4,2,46,11.307267,14.748609,0.434783
4,63127690860023ab6b50a0b2,4,46,13.690367,17.857000,0.608696
5,63254db937c17e8e9200fd54,2,46,17.288917,22.550761,0.608696
6,63474e67a5fd298c6103c409,1,46,19.257467,25.118435,0.630435
7,63d87d4ba0b86684507f201d,4,46,21.927500,28.601087,0.521739
8,647dfaca60dc6f90350e765b,0,46,24.947750,32.540543,0.739130
9,652ab3f37d5885fc2d4a9240,1,46,24.011850,31.319804,0.543478


In [376]:
# parition_length = 56
# def process_dataframe():
#     df = []
#     for participant in data:
#         session = participant['parsed_params']['SESSION_ID']

#         for qi in range(1, parition_length+1):
#             item = participant['order'][f'order{qi}']

#             if qi != 0:
#                 test_type = item['test_type'].split("_")[1]
#                 image_file = item['image_file']
#                 if test_type == 'calvi':
#                     if image_file[0] == "T":
#                         test_type = "calvi-trick"
#                     else:
#                         test_type = "calvi-standard"
#                 row = {
#                     'question': item['question'],
#                     'answer': item['answer'],
#                     'image_file': image_file,
#                     'test_type': test_type,
#                     'image_link': item['image_link'],
#                     'correct_answer': item['correct_answer'],
#                     'chart_type': item['chart_type'],
#                     'participant_id': session,
#                     'time_spent': (item['timestamp_end'] - item['timestamp_start']) / 1000,
#                     'order': qi
#                     # 'is_correct': int(item['answer'] == item['correct_answer'])
                    
#                 }
#                 df.append(row)
#             else:
#                 test_type = 'practice'
#                 image_file = item['image_link'].split("/")[-1]
#                 correct_answer = 2.22
#                 chart_type = "practice_pie"

    
#     return pd.DataFrame(df)

# df = process_dataframe()
# df['question_image'] = df['question'] + " + " + df['image_file']

# ## reformate correct answers
# orig_questions = []
# for test in ['ggr-mc', 'wainer', 'calvi-trick', 'calvi-standard', 'brbf', 'vlat']:
#     questions = pd.read_csv(f'https://data-visualization-benchmark.s3.us-west-2.amazonaws.com/{test}/questions.csv')
#     questions = questions.rename(columns={
#         'task_category': 'task_type'
#     })
#     questions['question_image'] = questions['question'] + " + " + questions['image_file']
#     questions = questions[['question_image', 'correct_answer', 'task_type']]
#     orig_questions.append(questions)
    
# orig_questions = pd.concat(orig_questions)
# df = df.rename(columns={
#     'correct_answer': 'raw_correct_answer',
# })
# df = pd.merge(df, orig_questions, on="question_image")
# df['correct_answer'] = df['correct_answer'].astype(str).replace({
#     "TRUE": "True",
#     "FALSE": "False"
# })
# def check_correct_answer(r):
#     if r['answer'] == "More in the spring" and r['correct_answer'] == "Spring":
#         r['correct_answer'] = 'More in the spring'
#     return r
# df = df.apply(check_correct_answer, axis=1)

# df['is_correct'] = df.apply(lambda item : int(item['answer'] == item['correct_answer']), axis=1)

In [377]:
df['test_type'].unique()

array(['wainer', 'brbf', 'ggr-mc', 'vlat', 'calvi', 'practice'],
      dtype=object)

In [378]:
# df.columns
# df['question_image'].nunique()

In [379]:
# print("Questions answered: ", len(df['question_image'].nunique()))

In [380]:
# distribution of test items
df[['test_type', 'participant_id']].value_counts().reset_index().sort_values(by="test_type")

,test_type,participant_id,count
19,brbf,67490943c58dd32e763b6c75,14
28,brbf,66be7f34388490c8fca8b77e,14
27,brbf,63127690860023ab6b50a0b2,14
26,brbf,65fb580ab7fbb2d518ee8eaa,14
25,brbf,647dfaca60dc6f90350e765b,14
...,...,...,...
112,wainer,60fd6618f9a377f54dcdf318,6
88,wainer,63d87d4ba0b86684507f201d,7
87,wainer,63127690860023ab6b50a0b2,7
103,wainer,66da38dc1d5b9365584508ca,6


## ITEM DF

In [381]:
# def find_word(r):
#     word = "above"
#     return (
#         str(r['mc1']).find(word) != -1 or 
#         str(r['mc2']).find(word) != -1 or 
#         str(r['mc3']).find(word) != -1 or 
#         str(r['mc4']).find(word) != -1 or 
#         str(r['mc5']).find(word) != -1 or 
#         str(r['mc6']).find(word) != -1 or 
#         str(r['mc7']).find(word) != -1 or 
#         str(r['mc8']).find(word) != -1 or 
#         str(r['mc9']).find(word) != -1
#     )
        
        

# item_df[item_df.apply(find_word, axis=1)].iloc[0]

## Participant Statistics

In [382]:
import math

In [383]:
np.corrcoef(df['is_correct'], df['time_spent'])

array([[ 1.        , -0.23421647],
       [-0.23421647,  1.        ]])

In [384]:
test_type_df = df.groupby(['test_type'])[['time_spent', 'is_correct']].mean()
print(np.corrcoef(df['is_correct'], df['time_spent'])[0][1])
print(np.corrcoef(test_type_df['is_correct'], test_type_df['time_spent'])[0][1])

-0.2342164720577249
-0.8511535757840848


In [385]:
# rt_df = df.groupby('test_type')[['is_correct', 'time_spent']].mean().reset_index()
rt_df = df.copy()
rt_df['time_spent'] = rt_df['time_spent'].apply(lambda x : math.floor(x / 10) * 10)
alt.Chart(rt_df).mark_line().encode(
    alt.X("time_spent:N", title="Response Times (s)"),
    alt.Y("mean(is_correct)", title="Accuracy"),
    color=alt.Color('test_type:N', scale=alt.Scale())
).properties(
    title="Binned Accuracy vs Participants' Response Times",
    width=600,
    height=400
)

alt.Chart(...)

In [386]:
# Aggregate data to calculate mean and confidence interval
agg_df = rt_df.groupby(['time_spent', 'test_type']).agg(
    mean_accuracy=('is_correct', 'mean'),
    count=('is_correct', 'size'),
    std=('is_correct', 'std')
).reset_index()

# Calculate standard error and 95% confidence intervals
agg_df['se'] = agg_df['std'] / (agg_df['count'] ** 0.5)
agg_df['ci_upper'] = agg_df['mean_accuracy'] + 1.96 * agg_df['se']
agg_df['ci_lower'] = agg_df['mean_accuracy'] - 1.96 * agg_df['se']

# Plot the line with error bands
line = alt.Chart(agg_df).mark_line().encode(
    alt.X("time_spent:N", title="Response Times (s)"),
    alt.Y("mean_accuracy:Q", title="Accuracy"),
    color=alt.Color('test_type:N', scale=alt.Scale())
)

error_band = alt.Chart(agg_df).mark_errorband().encode(
    alt.X("time_spent:N", title="Response Times (s)"),
    alt.Y("ci_lower:Q", title="Accuracy"),
    alt.Y2("ci_upper:Q"),
    color=alt.Color('test_type:N', scale=alt.Scale(), legend=None)  # Same color but no extra legend
)

# Combine line and error band
chart = (error_band + line).properties(
    title="Binned Accuracy vs Participants' Response Times",
    width=600,
    height=400
)

chart

alt.LayerChart(...)

In [387]:
 alt.Chart(df).mark_bar().encode(
    alt.X("time_spent:Q", bin=alt.Bin(step=10), title="Response Times (s)"),
    alt.Y("count()", title="Number of Items"),
    color=alt.Color('test_type:N', scale=alt.Scale())
).properties(
    title="Histogram of Participants' Response Times",
    width=600,
    height=400
)

alt.Chart(...)

In [388]:
df[['

SyntaxError: unterminated string literal (detected at line 1) (4165423909.py, line 1)

## Test Statistics 

In [389]:
df.groupby(['test_type'])['time_spent'].mean().reset_index().sort_values(by='time_spent', ascending=False)

,test_type,time_spent
1,calvi,26.303687
0,brbf,26.059912
4,vlat,24.008136
2,ggr-mc,21.891882
5,wainer,19.717006
3,practice,16.503849


In [306]:
df[df['is_correct'] == 0].groupby(['test_type'])['time_spent'].mean().reset_index().sort_values(by='time_spent', ascending=False)

,test_type,time_spent
0,brbf,32.131383
3,practice,27.275176
1,calvi,26.479593
4,vlat,26.018281
5,wainer,23.964442
2,ggr-mc,22.403240


In [307]:
# accuracy by test type
df.groupby(['test_type'])['is_correct'].mean().reset_index().sort_values(by='is_correct', ascending=False)

,test_type,is_correct
3,practice,0.855319
5,wainer,0.720779
0,brbf,0.653179
4,vlat,0.649606
2,ggr-mc,0.596774
1,calvi,0.422145


In [308]:
# df.groupby(['test_type'])['is_correct'].mean().reset_index().sort_values(by='is_correct', ascending=False)

### Checking BRBF test

In [309]:
brbf_questions = pd.read_csv('https://data-visualization-benchmark.s3.us-west-2.amazonaws.com/brbf/questions.csv')
brbf_questions['question_image'] = brbf_questions['question'] + " + " + brbf_questions['image_file']
brbf_df = df[(df['test_type'] == 'brbf')]
brbf_df = pd.merge(brbf_df, brbf_questions, on="question_image")

In [26]:
brbf_df.columns

Index(['question_x', 'answer', 'image_file_x', 'test_type_x', 'image_link_x',
       'raw_correct_answer', 'chart_type', 'participant_id', 'time_spent',
       'question_image', 'correct_answer_x', 'task_type_x', 'is_correct',
       'Unnamed: 0', 'correct_answer_y', 'image_file_y', 'mc1', 'mc10', 'mc11',
       'mc2', 'mc3', 'mc4', 'mc5', 'mc6', 'mc7', 'mc8', 'mc9', 'question_y',
       'question_type', 'task_type_y', 'test_type_y', 'image_link_y',
       'text_input_type', 'graph_type'],
      dtype='object')

In [27]:
# brbf_questions
brbf_df.groupby(['chart_type'])['is_correct'].mean().reset_index().sort_values(by='is_correct', ascending=False)

,chart_type,is_correct
0,bg,0.916667
1,bg-table,0.833333
2,lg1,0.833333
6,sp-table,0.833333
5,sp,0.750000
3,lg2,0.666667
4,lg2-table,0.583333


In [28]:
brbf_df.groupby(['chart_type'])['time_spent'].mean().reset_index().sort_values(by='time_spent', ascending=False)

,chart_type,time_spent
5,sp,38.939250
4,lg2-table,37.328417
6,sp-table,34.752333
2,lg1,33.036958
1,bg-table,30.191833
3,lg2,27.866083
0,bg,24.683958


In [29]:
brbf_df.groupby(['chart_type'])['is_correct'].mean().reset_index().sort_values(by='is_correct', ascending=False)

,chart_type,is_correct
0,bg,0.916667
1,bg-table,0.833333
2,lg1,0.833333
6,sp-table,0.833333
5,sp,0.750000
3,lg2,0.666667
4,lg2-table,0.583333


In [30]:
def print_info(r):
    # Extracting the relevant details
    question = r.get('question_x', 'N/A')
    image_link = r.get('image_link_x', 'N/A')
    correct_answer = r.get('correct_answer_x', 'N/A')
    answer = r.get('answer', 'N/A')
    time = r.get('time_spent', 'N/A')
    mc_options = [r.get(f'mc{i}', 'N/A') for i in range(1, 11)]
    
    # Printing the information
    print(f"Question: {question}")
    print(f"Image Link: {image_link}")
    print(f"Correct Answer: {correct_answer}")
    print(f"Participant Answer: {answer}")
    print(f"Time Taken: {time} seconds")
    print("Multiple Choice Options:")
    for i, option in enumerate(mc_options, start=1):
        print(f"  MC{i}: {option}")

incorr_brbf_df = brbf_df[(brbf_df['is_correct'] == 0)].sort_values(by="time_spent", ascending=False)
print_info(incorr_brbf_df.iloc[5])

Question: When did the highest increase in the unemployment trend occur on planet Nurag since the beginning of this time period?
Image Link: https://data-visualization-benchmark.s3.us-west-2.amazonaws.com/brbf/images/lg1-5.svg
Correct Answer: 11/01/06-05/01/10
Participant Answer: 05/01/10-11/01/13
Time Taken: 52.653 seconds
Multiple Choice Options:
  MC1: 01/01/00-05/01/03
  MC2: 05/01/03-11/01/06
  MC3: 11/01/06-05/01/10
  MC4: 05/01/10-11/01/13
  MC5: nan
  MC6: nan
  MC7: nan
  MC8: nan
  MC9: nan
  MC10: nan


## Time spent statistics

In [703]:
 df[df['is_correct'] == 0].sort_values(by=['time_spent'], ascending=True)[['correct_answer', 'answer']].iloc[0:25]

,correct_answer,answer
159,Cannot be inferred / Inadequate Information,No
62,9.4,8.7
168,False,True
218,75,73
156,Cannot be inferred / Inadequate Information,Yes
171,40,20
76,82.8,38.1
80,Thar,Hebron
24,$37.04 - $60.95,$35 - $65
41,None of the above.,Copies sold per year fell sharply in 2022.


In [704]:
avg_df = df[
(df['test_type'] == 'brbf')
].groupby(['question_image', 'correct_answer', 'answer'])[['is_correct', 'time_spent',]].mean().reset_index() 
avg_df = avg_df.replace({'calvi-trick': 'calvi', 'calvi-standard': 'calvi',})

avg_df = avg_df[avg_df['is_correct'] < 1].sort_values(by=['time_spent'], ascending=False)
avg_df

,question_image,correct_answer,answer,is_correct,time_spent
36,When did unemployment first go above 6.9 on pl...,01/01/09,07/01/08,0.0,59.779
39,When was unemployment at its lowest peak on pl...,01/01/08,07/01/08,0.0,52.423
32,What is the average adult literacy on the 5 pl...,82.9,90.1,0.0,51.909
17,On how many planets does high expenditure per ...,1,3,0.0,51.649
30,What is the approximate average life expectanc...,65.4,59.3,0.0,45.319
25,What has the general average unemployment rate...,6.6,4.8,0.0,44.905
26,What is the approximate average adult literacy...,79.9,70.3,0.0,40.705
33,What is the average adult literacy on the 5 pl...,91,87.7,0.0,38.078
15,How many planets spend approximately the same ...,1,5,0.0,37.779
14,How many planets spend approximately the same ...,2,1,0.0,34.913


In [705]:
# .head(10)[['correct_answer', 'answer']]
avg_df

,question_image,correct_answer,answer,is_correct,time_spent
36,When did unemployment first go above 6.9 on pl...,01/01/09,07/01/08,0.0,59.779
39,When was unemployment at its lowest peak on pl...,01/01/08,07/01/08,0.0,52.423
32,What is the average adult literacy on the 5 pl...,82.9,90.1,0.0,51.909
17,On how many planets does high expenditure per ...,1,3,0.0,51.649
30,What is the approximate average life expectanc...,65.4,59.3,0.0,45.319
25,What has the general average unemployment rate...,6.6,4.8,0.0,44.905
26,What is the approximate average adult literacy...,79.9,70.3,0.0,40.705
33,What is the average adult literacy on the 5 pl...,91,87.7,0.0,38.078
15,How many planets spend approximately the same ...,1,5,0.0,37.779
14,How many planets spend approximately the same ...,2,1,0.0,34.913


## Graph Statistics

In [706]:
chart_categories = {
    'Dot Plot': 'Dot Plot',
    'Line Chart': 'Line',
    'Pie Chart': 'Pie',
    'chart': 'Table',
    'radial': 'Radial',
    'line': 'Line',
    'bar': 'Bar',
    'Scatterplot': 'Scatter Charts',
    'Bar chart': 'Bar',
    'Stacked area chart': 'Stacked Area',
    '100% stacked bar chart': '100% Stacked Bar',
    'Line chart': 'Line',
    'Choropleth map': 'Geospatial',
    'Area chart': 'Area',
    'Choropleth Map': 'Geospatial',
    'Pie chart': 'Pie',
    'Stacked bar chart': 'Stacked Bar',
    'bg-table': 'Table',
    'bg': 'Bar',
    'sp-table': 'Table',
    'sp': 'Scatter',
    'lg2': 'Line',
    'lg1': 'Line',
    'lg1-table': 'Table',
    'lg2-table': 'Table',
    'Bar Chart': 'Bar',
    'Bubble Chart': 'Scatter',
    'Area Chart': 'Area',
    '100 % Stacked Bar Chart': '100% Stacked Bar',
    'Stacked Area Chart': 'Stacked Area',
    'Stacked Bar Chart': 'Stacked Bar',
    'Treemap': 'Treemap'
}
df['common_chart_type'] = df['chart_type'].replace(chart_categories)

In [707]:
# df.groupby(['common_chart_type'])['is_correct'].mean().reset_index()

In [708]:
# df.groupby(['test_type', 'common_chart_type'])['is_correct'].mean().reset_index().sort_values(by="is_correct")

## Task Type statistics

In [709]:
# df.groupby(['task_type'])['is_correct'].mean().reset_index().sort_values(by="is_correct")

In [710]:
# df.groupby(['test_type', 'task_type'])['is_correct'].mean().reset_index().sort_values(by="test_type")

In [711]:
# df.groupby(['test_type', 'task_type'])['is_correct'].mean().reset_index().sort_values(by="is_correct")

## Plots

In [712]:
def bootstrap_sampling(data, num_samples=1000):
    boot_means = []
    for _ in range(num_samples):
        sample = np.random.choice(data, size=len(data), replace=True)
        boot_means.append(sample.mean())
    return boot_means

def generate_accuracy_ci(data, category):
    data = data[['is_correct', category, 'question_image']].copy()
    data = data.rename(columns={
        category: 'category', 
        'question_image': 'item_id'
    })
    
    # Prepare data for bootstrap confidence intervals
    bootstrap_data = []
    for category, group in data.groupby('category'):
        boot_means = bootstrap_sampling(group['is_correct'])
        bootstrap_data.append(pd.DataFrame({
            'category': category,
            'boot_means': boot_means
        }))
    
    bootstrap_df = pd.concat(bootstrap_data)
    
    
    violin_chart = alt.Chart(bootstrap_df).transform_density(
        'boot_means',
        as_=['boot_means', 'density'],
        groupby=['category']
    ).mark_area(orient='horizontal').encode(
        x=alt.X('density:Q', title=None, stack='center', axis=alt.Axis(labels=False, ticks=True)),
        y=alt.Y('boot_means:Q', title='Bootstrapped Mean Accuracy'),
        color=alt.Color('category:N', title='Category'),
        column=alt.Column(
            'category:N',
            spacing=0,
            header=alt.Header(
                title=None,
                labelOrient='bottom',
                labelPadding=0
            )
        )
    ).configure_view(
        stroke=None,
    ).properties(width=100)
    
    return violin_chart

In [713]:
def bootstrap_sampling(data, num_samples=1000):
    boot_means = []
    for _ in range(num_samples):
        sample = np.random.choice(data, size=len(data), replace=True)
        boot_means.append(sample.mean())
    return boot_means

def generate_accuracy_ci(df, category):
    data = df[['is_correct', category, 'question_image']].copy()
    data = data.rename(columns={
        category: 'category', 
        'question_image': 'item_id'
    })

    item_level_data = df.rename(columns={
        category: 'category', 
        'question_image': 'item_id'
    }).groupby(['participant_id', 'category'])['is_correct'].mean().reset_index()
    
    # Prepare data for bootstrap confidence intervals
    results = []
    for cat, group in data.groupby('category'):
        boot_means = bootstrap_sampling(group['is_correct'])
        mean = np.mean(boot_means)
        ci_lower = np.percentile(boot_means, 2.5)
        ci_upper = np.percentile(boot_means, 97.5)
        results.append({
            'category': cat,
            'mean': mean,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper
        })
    
    ci_df = pd.DataFrame(results)
    
    # Create dot+error bar plot
    chart = alt.Chart(ci_df).mark_point(filled=True, size=50).encode(
        x=alt.X('category:N', title='Category'),
        y=alt.Y('mean:Q', title='Mean Accuracy'),
        color=alt.Color('category:N', legend=None)
    ) + alt.Chart(ci_df).mark_rule(strokeWidth=1.5).encode(
        x=alt.X('category:N'),
        y=alt.Y('ci_lower:Q'),
        y2='ci_upper:Q',
        color=alt.Color('category:N', legend=None)
    )

    scatter_plot = alt.Chart(item_level_data).mark_circle(size=16,opacity=0.8).encode(
        x=alt.X('category:N', title='Category'),
        y=alt.Y(f"is_correct:Q",),
        xOffset="jitter:Q",
        color=alt.Color('category:N', legend=None),
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())" 
    )
    
    return (chart + scatter_plot).properties(width=200, height=300, title=f"Accuracy for {category}")

In [714]:
# df.groupby(['question_image', 'test_type'])['is_correct'].mean().reset_index()
# df

In [715]:
generate_accuracy_ci(df.replace({'calvi-trick': 'calvi', 'calvi-standard': 'calvi'}), 'test_type')

alt.LayerChart(...)

In [44]:
# df[df['task_type'] == np.nan]

In [45]:
# generate_accuracy_ci(df, 'task_type').properties(width=400)

In [46]:
# generate_accuracy_ci(df, 'common_chart_type').properties(width=800)

In [47]:
test = 'ggr-mc'
alt.hconcat(
    generate_accuracy_ci(df[df['test_type'] == test], 'task_type').properties(title="Accuracy by Task Type"),
    generate_accuracy_ci(df[df['test_type'] == test], 'chart_type').properties(title="Accuracy by Chart Type")
).properties(title="GGR")

alt.HConcatChart(...)

In [48]:
df[df['test_type'] == test].groupby(['test_type'])['time_spent'].mean().reset_index().sort_values(by='time_spent', ascending=False)

,test_type,time_spent
0,ggr-mc,30.224769


In [49]:
test = 'vlat'
alt.hconcat(
    generate_accuracy_ci(df[df['test_type'] == test], 'task_type').properties(title="Accuracy by Task Type"),
    generate_accuracy_ci(df[df['test_type'] == test], 'chart_type').properties(title="Accuracy by Chart Type")
).properties(title=test)

alt.HConcatChart(...)

In [65]:
test = 'brbf'
alt.hconcat(
    generate_accuracy_ci(df[df['test_type'] == test], 'task_type').properties(title="Accuracy by Task Type"),
    generate_accuracy_ci(df[df['test_type'] == test], 'chart_type').properties(title="Accuracy by Chart Type")
).properties(title=test)

alt.HConcatChart(...)

In [64]:
test = 'calvi'
calvi_df = df.replace({'calvi-trick': 'calvi', 'calvi-standard': 'calvi'})
alt.hconcat(
    generate_accuracy_ci(calvi_df[calvi_df['test_type'] == test], 'task_type').properties(title="Accuracy by Task Type"),
    generate_accuracy_ci(calvi_df[calvi_df['test_type'] == test], 'chart_type').properties(title="Accuracy by Chart Type", width=600)
).properties(title=test)

alt.HConcatChart(...)

In [52]:
test = 'calvi-standard'
alt.hconcat(
    generate_accuracy_ci(df[df['test_type'] == test], 'task_type').properties(title="Accuracy by Task Type"),
    generate_accuracy_ci(df[df['test_type'] == test], 'chart_type').properties(title="Accuracy by Chart Type")
).properties(title=test)

alt.HConcatChart(...)

In [53]:
test = 'calvi-trick'
alt.hconcat(
    generate_accuracy_ci(df[df['test_type'] == test], 'task_type').properties(title="Accuracy by Task Type"),
    generate_accuracy_ci(df[df['test_type'] == test], 'chart_type').properties(title="Accuracy by Chart Type")
).properties(title=test)

alt.HConcatChart(...)

In [54]:
df[df['test_type'] == 'wainer'][['question_image']].value_counts()

question_image                                                                          
How many months have more rain than the average month? + bar.png                            4
Which season has more rain, the summer or the spring? + line.png                            4
How much does it rain in March? + table.png                                                 3
How many months have less than 40mm of rain? + radial.png                                   3
Which month has 25 mm of rain? + bar.png                                                    3
In which season does each month have less rain than the month before? + table.png           3
Which season has the most rain? + bar.png                                                   3
How much does it rain in March? + radial.png                                                3
In San Francisco, as the weather gets warmer, there is generally more rain. + line.png      3
In San Francisco, as the weather gets warmer, there is generally 

In [55]:
test = 'wainer'
alt.hconcat(
    generate_accuracy_ci(df[df['test_type'] == test], 'task_type').properties(title="Accuracy by Task Type"),
    generate_accuracy_ci(df[df['test_type'] == test], 'chart_type').properties(title="Accuracy by Chart Type")
).properties(title=test)

alt.HConcatChart(...)

In [56]:
df[df['test_type'] == 'calvi-standard'].groupby(['common_chart_type'])['is_correct'].mean().reset_index()

,common_chart_type,is_correct
0,N-100% stacked bar chart,0.50
1,N-Area chart,1.00
2,N-Bar chart,1.00
3,N-Choropleth map,1.00
4,N-Line chart,0.25
5,N-Pie chart,0.50
6,N-Scatterplot,1.00
7,N-Stacked bar chart,0.50


In [57]:
df[df['test_type'] == 'calvi-standard'].groupby(['task_type'])['is_correct'].mean().reset_index()

,task_type,is_correct
0,Find Correlations/Trends,0.60
1,Make Comparisons,0.75


In [95]:
df = df.replace({'calvi-trick': 'calvi', 'calvi-standard': 'calvi'})
df['test_type'].unique()

array(['ggr-mc', 'vlat', 'brbf', 'wainer', 'calvi'], dtype=object)

In [717]:
def create_pairwise_agent_heatmap(df, x, y, domain, units_of_measure, include_text=True, text_format=".3f"):

    base = alt.Chart(df).mark_rect().encode(
        x=alt.X(x, scale=alt.Scale(domain=domain)),  
        y=alt.Y(y, scale=alt.Scale(domain=domain)),
    )

    color_domain=[-1,1]
    color_reverse = False
    color_condition = alt.condition(
        alt.datum[units_of_measure] < 0.5,
        alt.value('black'),
        alt.value('white')
    )


    height=300
    width=400 
    title=f"Accuracy Vector Correlation between Participants"

    heatmap = base.mark_rect().encode(
        color=alt.Color(f'{units_of_measure}:Q', 
                        legend=None, 
                        scale=alt.Scale(
                            scheme="redblue",
                            domain=color_domain,
                            reverse=color_reverse)
                       ),
    )

    text = base.mark_text(baseline='middle').encode(
        alt.Text(f'{units_of_measure}:Q', format=text_format),
        color=color_condition
    )

    chart = heatmap.properties(
        width=width,
        height=height,
        title=title
    ) 
    if include_text:
        chart = chart + text

    return chart

## relationship between performance on pairs of tests, within a participant

In [718]:
# data = df.groupby(['participant_id', 'test_type', ]).mean('is_correct').reset_index()
# data = data.pivot(index='test_type', columns='participant_id', values='is_correct').reset_index()
# participants = df['participant_id'].unique()

# corr_df = []
# for p1 in participants:
#     for p2 in participants:
#         corr = np.corrcoef(data[p1], data[p2])[0][1]
#         corr_df.append({
#             'corr': corr,
#             'participant1': p1,
#             'participant2': p2
#         })
# corr_df = pd.DataFrame(corr_df)
# create_pairwise_agent_heatmap(corr_df, x='participant1', y='participant2', domain=participants, units_of_measure='corr')

In [719]:
# df

## to what degree participants who are doing well on one test are also doing well on other tests

In [724]:
tests = ['ggr-mc', 'vlat', 'brbf', 'wainer', 'calvi']
data = df.groupby(['participant_id', 'test_type', ]).mean('is_correct').reset_index()
data = data.pivot(index='participant_id', columns='test_type', values='is_correct').reset_index()

corr_df = []
for t1 in tests:
    for t2 in tests:
        corr = np.corrcoef(data[t1], data[t2])[0][1]
        corr_df.append({
            'corr': corr,
            'test1': t1,
            'test2': t2
        })
corr_df = pd.DataFrame(corr_df)
create_pairwise_agent_heatmap(corr_df, x='test1', y='test2', domain=tests, units_of_measure='corr')
# scatter = alt.Chart(data).mark_cir/cle().encode(
#     x=alt.X("calvi", type='quantitative'),
#     y=alt.Y("vlat", type='quantitative')
# )


alt.LayerChart(...)

In [723]:
def create_pairwise_agent_scatterplot(df, x, y, domain):

    charts = []
    for d1 in domain:
        chart_row = []
        for d2 in domain:
            scatter = alt.Chart(df).mark_circle(size=50).encode(
                y=alt.Y(d1, type='quantitative', title=d1, scale=alt.Scale(domain=[0,1])),
                x=alt.X(d2, type='quantitative', title=d2, scale=alt.Scale(domain=[0,1])),
                color=alt.Color('participant_id')
            ).properties(width=50, height=50)
            chart_row.append(scatter)
        charts.append(chart_row)

    return alt.vconcat(
        *[alt.hconcat(*chart_row) for chart_row in charts]
    )


create_pairwise_agent_scatterplot(data, x='test1', y='test2', domain=tests)

alt.VConcatChart(...)

##  any relationship between RT and accuracy

In [67]:
np.corrcoef(df['time_spent'], df['is_correct'])

array([[ 1.        , -0.21724949],
       [-0.21724949,  1.        ]])

In [68]:
# len(df)

In [69]:
data = df.groupby(['participant_id']).agg({
    'is_correct': 'mean',  
    'time_spent': 'mean'
}).reset_index()
np.corrcoef(data['time_spent'], data['is_correct'])[0][1]

-0.2159281128174197

In [82]:
# df.groupby(['participant_id', 'test_type']).agg({
#     'is_correct': 'mean',  
#     'order': 'mean'
# })

In [93]:
# df.groupby(['participant_id', 'test_type']).agg({
#     'is_correct': 'mean',  
#     'order': 'mean'
# }).reset_index()

In [100]:
data = df.groupby(['participant_id', 'test_type']).agg({
    'is_correct': 'mean',  
    'order': 'min',
    'time_spent': 'mean'
}).reset_index().replace({})
print(np.corrcoef(data['order'], data['is_correct'])[0][1])
alt.hconcat(
alt.Chart(data).mark_circle(size=50).encode(
    x=alt.X("order", type='quantitative', title='Test Start Order'),
    y=alt.Y("is_correct", type='quantitative', title='Participant Test Accuracy'),
    color='test_type'
),
alt.Chart(data).mark_circle(size=50).encode(
    x=alt.X("order", type='quantitative', title='Test Start Order'),
    y=alt.Y("time_spent", type='quantitative', title='Participant Averge Time Spent'),
    color='test_type'
),
)

-0.45272926688551035


alt.HConcatChart(...)

In [70]:
data = df.groupby(['participant_id', 'test_type']).agg({
    'is_correct': 'mean',  
    'time_spent': 'mean'
}).reset_index()

corr = np.corrcoef(data['time_spent'], data['is_correct'])[0][1]
print(corr)
alt.Chart(data).mark_circle(size=50).encode(
    x=alt.X("time_spent", type='quantitative', title='Participant Averge Time Spent'),
    y=alt.Y("is_correct", type='quantitative', title='Participant Test Accuracy'),
    color='test_type'
)

-0.3222122899816284


alt.Chart(...)

In [ ]:
# np.corrcoef(df['time_spent'], df['is_correct'])
# df
# data = df.groupby(['participant_id', 'test_type']).sum(['is_correct', 'time_spent']).reset_index()

# df[(df['test_type'] == 'vlat') & (df['is_correct'] == 0)]

In [ ]:
tests = ['ggr-mc', 'vlat', 'brbf', 'wainer', 'calvi']
data = df.groupby(['participant_id', 'test_type', ]).mean('is_correct').reset_index()
data = data.pivot(index='participant_id', columns='test_type', values='is_correct').reset_index()

corr_df = []
for t1 in tests:
    for t2 in tests:
        corr = np.corrcoef(data[t1], data[t2])[0][1]
        corr_df.append({
            'corr': corr,
            'test1': t1,
            'test2': t2
        })
corr_df = pd.DataFrame(corr_df)
create_pairwise_agent_heatmap(corr_df, x='test1', y='test2', domain=tests, units_of_measure='corr')

In [ ]:
alt.Chart(df).mark_circle().encode(
    x=alt.X("time_spent", type='quantitative'),
    y=alt.Y("is_correct", type='quantitative')
)

 are trials where a given participant takes longer more likely to be correct than trials where that same participant takes less long?/

In [ ]:
# df

In [1573]:
import pandas as pd
import numpy as np
import altair as alt
from joblib import Parallel, delayed
from numpy.random import Generator, PCG64
rng = np.random.default_rng()

def bootstrap_ci(
        data,
        measure,
        id_col,
        n_iterations=1000,
        statistic=np.mean):
    
    items = list(data[id_col].unique())
    n_size = len(items)
    df = data.copy()

    def bootstrap_iteration(data, chosen_items):
        filter_df = data[data[id_col].isin(chosen_items)] # Filter based on chosen questions
        bs_mean = statistic(filter_df[measure]) 
        return (bs_mean, list(chosen_items))

    qset_means = Parallel(n_jobs=-1)(
        delayed(bootstrap_iteration)(
            df.copy(),
            rng.choice(items, n_size,  replace=True)
        ) for _ in range(n_iterations)
    )
    
    means = []
    qs_used = []
    means = [bs_mean for bs_mean, chosen_qs in qset_means]
 
    # 95% confidence interval
    lower = np.percentile(means, 2.5)
    upper = np.percentile(means, 97.5)
    
    return lower, upper


def create_confidence_interval_df(
    data,
    measure, 
    id_col,
    condition_col,
    statistic=np.mean
):
    data_list = []

    for condition in data[condition_col].unique():
        condition_data = data[data[condition_col] == condition]

        lower, upper = bootstrap_ci(condition_data, measure=measure, statistic=statistic, id_col=id_col)

        data_list.append({
            "condition": condition,
            "ci_upper": upper, 
            "ci_lower": lower,
        })

    return pd.DataFrame(data_list)

def get_error_dfs(df, measure, item_level_col):
    agg_data = df.groupby(['condition']).agg(
        mean_prop=(measure, 'mean')
    ).reset_index()
    
    word_level_data = df.groupby(['condition', item_level_col]).agg(
        mean_prop=(measure, 'mean')
    ).reset_index()
    word_level_data['item_id'] = word_level_data['condition'] + word_level_data[item_level_col]
    
    ci_df = create_confidence_interval_df(
        data=df,
        measure=measure,
        id_col=item_level_col,
        condition_col='condition'
    )

    error_df = pd.merge(agg_data, ci_df, on=['condition'])

    return error_df, word_level_data

def create_error_plot(error_df, item_level_df, condition_domain, color_domain):
    ci_plot = alt.Chart(error_df).mark_errorbar().encode(
        x=alt.X("condition:N", title=None),
        y=alt.Y("ci_upper", title="Proportion of Detail Pen Selections"),
        y2=alt.Y2("ci_lower"),
        strokeWidth=alt.value(2),
        color=alt.Color('condition').legend(None)
    )

    mean_points = alt.Chart(error_df).mark_point(filled=True, size=75, opacity=1).encode(
        x=alt.X('condition:N', scale=alt.Scale(domain=condition_domain)),
        y=alt.Y('mean_prop:Q', scale=alt.Scale(domain=[0,1])),
        color=alt.Color('condition:N', scale=alt.Scale(domain=condition_domain, range=color_domain)),
    )
    
    scatter_plot = alt.Chart(item_level_df).mark_circle(size=16,opacity=0.5).encode(
        x=alt.X("condition:N", title=None),
        y=alt.Y("mean_prop:Q", scale=alt.Scale(domain=[0,1])),
        xOffset="jitter:Q",
        color=alt.Color('condition:N').legend(None),
    ).transform_calculate(
        jitter="sqrt(-2*log(random()))*cos(2*PI*random())" 
    )
    
    
    return mean_points + ci_plot + scatter_plot

df = pd.DataFrame({
    'participant_id': np.repeat(range(1, 6), 12),  # 10 participants, repeated for 12 items
    'item_id': np.tile(range(1, 13), 5),          # 12 items, repeated for each participant
    'condition': np.random.choice(['ConditionA', 'ConditionB', 'ConditionC', 'ConditionD'], 60),
    'is_correct': np.random.randint(0, 2, 60)     # Measure of accuracy (random 0 or 1)
})
df['item_id'] = df['item_id'].astype(str)
df['participant_id'] = df['participant_id'].astype(str)

error_df, item_df = get_error_dfs(
        df,
        measure='is_correct',
        item_level_col='item_id'
)

condition_domain = ['ConditionA', 'ConditionB', 'ConditionC', 'ConditionD']
color_domain = ["#457b9d", "#1d3557", "#f4a261", "#e76f51"]


plot = create_error_plot(
    error_df, 
    item_df,
    condition_domain=condition_domain,
    color_domain=color_domain
)
plot = plot.properties(width=200, title=f"Bootstrapping over items")
plot

alt.LayerChart(...)